# Normalizer

This is a component that normalize samples individually to unit norm using an implementation from [Scikit-learn](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.Normalizer.html). 
<br>
Scikit-learn is an open source machine learning library that supports supervised and unsupervised learning. It also provides various tools for model fitting, data preprocessing, model selection and evaluation, and many other utilities.

This notebook shows:
- how to use SDK to load the dataset and save a model.
- how to receive parameters from the platform.

In [ ]:
dataset = "boston" #@param {type:"string"}
out_dataset = "boston-normalized" #@param {type:"string"}
target = "medv" #@param {type:"string"}
experiment_id = "e6c07a4e-4902-43c3-b02f-efe8f051cb0c" #@param {type:"string"}
operator_id = "c0deb81a-540e-4d51-bf8f-c332f9b9fd73" #@param {type:"string"}

## Load dataset

Import and put the whole dataset in a pandas.DataFrame.

In [ ]:
from platiagro import load_dataset

df = load_dataset(name=dataset)
X = df.drop(target, axis=1).to_numpy()
y = df[target].to_numpy()

## Load metadata about the dataset
For example, below we get the feature type for each column in the dataset. (eg. categorical, numerical, or datetime)

In [ ]:
from platiagro import stat_dataset
from platiagro.featuretypes import infer_featuretypes

try:
    metadata = stat_dataset(name=dataset)
    featuretypes = metadata["featuretypes"]
except KeyError:
    featuretypes = infer_featuretypes(df)

## Fit a model using sklearn.preprocessing.Normalizer

In [ ]:
from platiagro.featuretypes import NUMERICAL
from sklearn.preprocessing import Normalizer

target_idx = df.columns.tolist().index(target)
# selects the numerical features
numerical_idxs = [idx for idx, ft in enumerate(featuretypes) if ft == NUMERICAL and idx != target_idx]
estimator = Normalizer()

if len(numerical_idxs) > 0:
    X[:, numerical_idxs] = estimator.fit_transform(X[:, numerical_idxs])

## Save dataset

Stores the transformed dataset in a object storage.<br>

In [ ]:
from platiagro import save_dataset

save_dataset(name=out_dataset, df=df)

## Save model

Stores the model artifacts in a object storage.<br>
It will make the model available for future deployments.

In [ ]:
from platiagro import save_model

save_model(experiment_id=experiment_id, model={"estimator": estimator})